In [143]:
import pandas as pd
import requests
import json
gameweek_to_predict = 2
folder_to_save_to = r"C:\Users\Jim\Documents\fantasy prediction/"
n_players_to_pick = 12

In [144]:
#Get list of fpl player names and ids
fpl_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response=requests.get(fpl_url)
fpl_response = json.loads(response.content)

players = fpl_response['elements']
players_df_whole = pd.DataFrame.from_dict(players)
players_df = players_df_whole[['team', 'web_name', 'id', 'chance_of_playing_next_round','now_cost', 'minutes', 'first_name', 'second_name', 'element_type']]
players_df.dropna()

teams = fpl_response['teams']
teams_df_whole = pd.DataFrame.from_dict(teams)
teams_df = teams_df_whole[['name', 'id']]
players_df = pd.merge(players_df, teams_df, how='left', left_on = ['team'], right_on = ['id'])
players_df['team_name'] = players_df['name']
players_df = players_df[['team', 'web_name', 'id_x', 'chance_of_playing_next_round','now_cost', 'minutes', 'team_name', 'first_name', 'second_name', 'element_type']]
players_df['full_name'] = players_df['first_name'] + players_df['second_name']

In [145]:
predictions = pd.read_csv(folder_to_save_to + "point_prediction_gw" + str(gameweek_to_predict) + ".csv")

predictions['prediction'] = predictions.iloc[:, 1].fillna(value=0)
predictions.drop(predictions.index[11], inplace = True)
predictions.drop(predictions.index[0], inplace = True)
predictions = predictions[predictions.iloc[:, 0] > 2]

predictions_fpl = pd.merge(predictions, players_df, how = 'left', left_on = ['name'], right_on = ['full_name'])
predictions_fpl['chance_of_playing_next_round'].fillna(value=100, inplace=True)
predictions_fpl['playing_points'] = predictions_fpl['prediction']*predictions_fpl['chance_of_playing_next_round']
predictions_fpl.reset_index(inplace=True)

In [146]:
predictions_fpl = pd.merge(predictions, players_df, how = 'left', left_on = ['name'], right_on = ['full_name'])

predictions_fpl['chance_of_playing_next_round'].fillna(value=100, inplace=True)
predictions_fpl['playing_points'] = predictions_fpl['prediction']*predictions_fpl['chance_of_playing_next_round']
predictions_fpl.reset_index(inplace=True)



In [147]:
#a quick function to get all possible formations allowed in FPL
def get_formations(n_players=11):
    possible_formations = []
    for DEF in range(6):
        for MID in range(6):
            for FWD in range(4):
                formation = [DEF, MID, FWD]
                if sum(formation) == (n_players-1):
                    possible_formations.append(formation)
    return possible_formations

[[3, 5, 3], [4, 4, 3], [4, 5, 2], [5, 3, 3], [5, 4, 2], [5, 5, 1]]


In [190]:
#pick the highest predicted scoring team for the gameweek
possible_formations = get_formations(n_players_to_pick)
available_money = 1000 - ((15 - (n_players_to_pick)) * 45)
price_weighting = 0
cost = []
team_points = []
for formation in possible_formations:
    while sum(cost) > available_money or sum(cost) ==0:
        picks = []
        points = []
        cost = []
        team = []
        position = []
        predictions_fpl['selection_value'] = (predictions_fpl['playing_points'] / predictions_fpl['now_cost']) + (price_weighting/predictions_fpl['now_cost'])
        predictions_fpl.sort_values(['selection_value'], ascending = False, inplace= True)
        predictions_fpl = predictions_fpl.reset_index(drop=True)
        for i in range(len(predictions_fpl)):
            vc = 0
            for ele in position:
                if (ele == predictions_fpl.loc[i, 'element_type']):
                    vc = vc + 1

            tvc = 0
            for ele in team: 
                if (ele == predictions_fpl.loc[i, 'team']): 
                    tvc = tvc + 1


            if (tvc < 3) and ((predictions_fpl.loc[i, 'element_type'] == 1 and vc < 1) or (predictions_fpl.loc[i, 'element_type'] == 2 and vc < formation[0]) or (predictions_fpl.loc[i, 'element_type'] == 3 and vc < formation[1]) or (predictions_fpl.loc[i, 'element_type'] == 4 and vc <formation[2])):
                picks.append(predictions_fpl.loc[i, 'name'])
                points.append(predictions_fpl.loc[i, 'prediction'])
                cost.append(predictions_fpl.loc[i, 'now_cost'])
                position.append(predictions_fpl.loc[i, 'element_type'])
                team.append(predictions_fpl.loc[i, 'team'])
        price_weighting +=1
    if sum(points) > sum(team_points):
        team_points = points
        team_picks = picks
        team_cost = cost
        final_formation = formation
print(team_picks)
print(sum(team_cost))
print(sum(team_points))
print(final_formation)
dream_team = pd.DataFrame(team_picks, columns = ['name'])
dream_team['prediction'] = team_points
dream_team.to_csv(folder_to_save_to + "dream_team_gw" + str(gameweek_to_predict) + ".csv")


['AndrewRobertson', 'GeorgeBaldock', 'EzriKonsa Ngoyo', 'MasonGreenwood', 'MathewRyan', 'ConorHourihane', 'JuanMata', 'JamesMilner', 'JohnLundstram', 'AlexandreLacazette', 'SergioAgüero', 'JamieVardy']
810
80.68323194980621
[3, 5, 3]


In [149]:
#function to make one (highest_valued) transfer
def pick_best_transfer(prev_gw_name_col, prev_gw_position_col, prev_gw_price_col,  prediction_col, pred_price_col, pred_position_col, pred_name_col):
    money_ITB = (1000 - ((15 - (len(prev_gw_price_col))) * 45)  - sum(prev_gw_price_col))
    transfers = []
    transfer_points = []
    prev_next_prediction = []
    for i in range(len(prev_gw_name_col)):
        available_money = prev_gw_price_col[i] + money_ITB
        best_transfer = prev_gw_name_col[i]
        i_next_prediction = prediction_col[(pred_name_col[pred_name_col == best_transfer].index)]
        i_next_prediction = prediction_col[(pred_name_col[pred_name_col == best_transfer].index)]
        prev_next_prediction.append(sum(i_next_prediction))
        best_transfer_points = sum(i_next_prediction)
        for i2 in range(len(prediction_col)):
            if (pred_price_col[i2] <= available_money) and (prediction_col[i2] > best_transfer_points) and (prev_gw_position_col[i] == pred_position_col[i2]) and (pred_name_col[i2] not in prev_gw_name_col.values):
                best_transfer = pred_name_col[i2]
                best_transfer_points = prediction_col[i2]
        transfers.append(best_transfer)
        transfer_points.append(best_transfer_points)
    potential_transfers = pd.DataFrame(prev_gw_name_col, columns = ['name'])
    potential_transfers['player_points'] = prev_next_prediction
    potential_transfers['position'] = prev_gw_position_col
    potential_transfers['transfer'] = transfers
    potential_transfers['points'] = transfer_points
    potential_transfers['points_gain'] = potential_transfers['points'] - potential_transfers['player_points']
    print(potential_transfers)
    best_transfer = potential_transfers.sort_values(['points_gain'], ascending = False).iloc[0]
    best_transfer = potential_transfers.sort_values(['points_gain'], ascending = False).iloc[0]
    print("best transfer this week: in = " + best_transfer['transfer'] + " out = " + best_transfer['name'] + " gain =" + str(best_transfer['points_gain']))           
    next_gw_picks = pd.DataFrame((potential_transfers['name'][potential_transfers['name'] != best_transfer['name']]), columns= ['name'])
    next_gw_picks = next_gw_picks['name'].append(pd.DataFrame([best_transfer['transfer']]), ignore_index = True)
    print(next_gw_picks)
    return potential_transfers, best_transfer, next_gw_picks
    

In [192]:
#get picks from week before + update the actual points scored on them
prev_gw_picks = pd.read_csv(folder_to_save_to + "gw" + str(gameweek_to_predict-1)+ "_picks.csv")
last_gw_points = []
for id_number in prev_gw_picks['id_x']:
    in_fpl_url="https://fantasy.premierleague.com/api/element-summary/" + str(id_number) + "/"
    fpl_player = requests.get(in_fpl_url)
    fpl_player = json.loads(fpl_player.content)
    fpl_df_x = pd.DataFrame.from_dict(fpl_player['history'])
    if len(fpl_df_x) == (gameweek_to_predict -1):
        player_points_last_gw = fpl_df_x['total_points'].iloc[0]
        last_gw_points.append(player_points_last_gw)
    else:
        last_gw_points.append(np.nan)
prev_gw_picks['actual_points'] = last_gw_points
prev_gw_picks.to_csv(folder_to_save_to + "gw" + str(gameweek_to_predict-1)+ "_picks.csv")

In [151]:
#run the transfer funtion
transfer_options, best_transfer, next_gw_picks = pick_best_transfer(prev_gw_picks['name'], prev_gw_picks['element_type'], prev_gw_picks['now_cost'], predictions_fpl['prediction'], predictions_fpl['now_cost'], predictions_fpl['element_type'], predictions_fpl['name'])

                        name  player_points  position  \
0   Pierre-EmerickAubameyang      11.314894         3   
1               MohamedSalah      11.745067         3   
2                MattDoherty       5.563466         2   
3            AndrewRobertson      10.532272         2   
4         AlexandreLacazette       7.308757         4   
5             Virgilvan Dijk       7.971308         2   
6              GeorgeBaldock       7.227848         2   
7                EndaStevens       6.184373         2   
8              JohnLundstram       4.511487         3   
9            SébastienHaller       3.148787         4   
10           LukaszFabianski       4.202721         1   
11               GranitXhaka       3.851213         3   

                    transfer     points  points_gain  
0   Pierre-EmerickAubameyang  11.314894     0.000000  
1               MohamedSalah  11.745067     0.000000  
2                 LucasDigne   7.266125     1.702659  
3            AndrewRobertson  10.53227

In [153]:
#save all the neccessary outputs for tableau as files
next_gw_picks_fpl = pd.merge(next_gw_picks, predictions_fpl, how = 'left', left_on = [0], right_on = ['full_name'])
gameweek = []
for i in range(len(next_gw_picks_fpl)):
    gameweek.append(gameweek_to_predict)
next_gw_picks_fpl['gameweek'] = gameweek
net_gw_picks_fpl['actual_points'] = np.nan
next_gw_picks_fpl.to_csv(folder_to_save_to + "gw" + str(gameweek_to_predict) + "_picks.csv")

transfer_options.to_csv(folder_to_save_to + "gw" + str(gameweek_to_predict) + "_transfer_options.csv")
best_transfer.to_csv(folder_to_save_to + "gw" +str(gameweek_to_predict) + "_best_transfer.csv")

C:\Users\Jim\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """
